In [1]:
from sys import path
from pathlib import Path

import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format

# Se genera la ruta absoluta al directorio "Comisiones"
path_package = str(Path.cwd().parent)

# Se agrega la ruta al directorio sys.path
path.append(path_package)


from algoritmos.algoritmo_cobranza_mes import cobranza_mes_func

from descriptions.descriptions import descrip_sales_users_df, descrip_product_df, descrip_partner_df

In [ ]:
cobranza_enero = cobranza_mes_func(1)
cobranza_febrero = cobranza_mes_func(2)
cobranza_marzo = cobranza_mes_func(3)
cobranza_abril = cobranza_mes_func(4)
cobranza_mayo = cobranza_mes_func(5)

cobranza_año = pd.concat([cobranza_enero, cobranza_febrero, cobranza_marzo, cobranza_abril, cobranza_mayo], ignore_index=True)

In [ ]:
cobranza_año['Mes'] = cobranza_año['pay_date'].dt.month
cob_group_mes = cobranza_año[['Mes', 'pay_amount']].groupby('Mes').sum().reset_index()